<h1> Systems in pyMOR </h1>

<br/>

<font size="4">

  - `pymor.models.iosys` consists of base classes for system-theoretic models
      - `LTIModel`
      - `SecondOrderModel`
      - ...
  - Associated transfer function models are in `pymor.models.transfer_function`
  - Problem-specific models can be implemented
    
</font>

<h1> Triple Chain Oscillator </h1>

<img src="triple_chain_oscillator_ctrl_obs.png" width="750">

<font size="4">

  - $3$ rows of $l$ masses $m_1, m_2, m_3$ 
      - connected via springs to mass $m_0$
      - fixed on left-hand side
  - $k_0, k_1, k_2, k_3$ indicate stiffness of springs
  - dampers on right-hand side have viscosity $\nu_1, \nu_2, \nu_3$
  - force applied to mass $m_0$ via control $u(t)$ 
  - measure displacement of masses directly connected to dampers

</font>

<h1> Parametric second-order system </h1>

<br/>

<font size="4">
    <br/>
    \begin{align}
        M \ddot{x}(t)
            + E(\alpha, \beta) \dot{x}(t)
            + K x(t)
            & =
                B u(t), \\
            y(t)
            & =
                C x(t)
    \end{align}
    <br/>

  - state $x(t) \in \mathbb{R}^n$ indicates displacement of the $n = 3l + 1$ masses
  - mass matrix $M \in \mathbb{R}^{n \times n}$, damping matrix $E(\alpha, \beta) \in \mathbb{R}^{n \times n}$, stiffness matrix $K \in \mathbb{R}^{n \times n}$
      - consider Rayleigh damping with parameters $\alpha, \beta \in \mathbb{R}$ such that $E(\alpha, \beta) = E_0 + \alpha M + \beta K$
      - parameter pair of interest $(\alpha, \beta) = (0.2, 0.2)$
      - $E = E(0.2, 0.2)$
  - input $u(t) \in \mathbb{R}$: actuating force applied to $m_0$
  - measurement $y(t) \in \mathbb{R}^3$: displacement of three masses directly connected to dampers    
  - see [N. Truhar, K. Veselić [2009]](https://epubs.siam.org/doi/pdf/10.1137/070683052?casa_token=Q_JwN-uMB_wAAAAA:6BM_3mfh1WfpG9TP8MJ_iUtX3dYGz5gEf-OYlTlFA0ByC0LnbYXlssmo9R8J7A-XiMK8MA0QzK4e) for the original model
</font>

<h1> Reducing the model with pyMOR </h1>
<br/>

<font size="4">

1. Get Numpy/Scipy objects for all system matrices.
2. Build a `SecondOrderModel` using `NumpyMatrixOperators`.
3. Choose a reductor from `pymor.models.reductors` that can be used for the model.
4. Compute ROM.

</font>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sps
from pymor.basic import *

plt.rcParams['axes.grid'] = True

# number of rows
l = 50

# number of masses
n = 3*l+1

# masses
m1 = 1
m2 = 2
m3 = 3
m0 = 10

# spring stiffness
k1 = 10
k2 = 20
k3 = 1
k0 = 50

# damper viscosity
v = 5

M = sps.spdiags(np.concatenate([m1*np.ones(l), m2*np.ones(l), m3*np.ones(l), [m0]]), 0, n, n)

Ki = sps.diags([-np.ones(l-1), 2*np.ones(l), -np.ones(l-1)], [-1, 0, 1])

K = sps.lil_matrix((n, n))
K[:3*l, :3*l] = sps.block_diag([k1*Ki, k2*Ki, k3*Ki])

el = sps.lil_matrix((l, 1))
el[-1] = 1

K[:3*l, -1] = sps.vstack([-k1*el, -k2*el, -k3*el])
K[-1, :3*l] = K[:3*l, -1].T

K[-1, -1] = k1 + k2 + k3 + k0

E0 = sps.lil_matrix((n, n))
E0[0, 0] = E0[l, l] = E0[2*l, 2*l] = v

B = np.zeros((n, 1))
B[-1] = 1

C = np.zeros((3, n))
C[0, 0] = C[1, l] = C[2, 2*l] = 1

In [2]:
from pymor.models.iosys import SecondOrderModel

Mop = NumpyMatrixOperator(M)
Kop = NumpyMatrixOperator(K)
E0op = NumpyMatrixOperator(E0)
Bop = NumpyMatrixOperator(B)
Cop = NumpyMatrixOperator(C)

Eop = E0op + ProjectionParameterFunctional('a') * Mop + ProjectionParameterFunctional('b') * Kop

pfom = SecondOrderModel(Mop, Eop, Kop, Bop, Cop)

print(pfom.parameters)

{a: 1, b: 1}


In [3]:
print(pfom)

SecondOrderModel
    class: SecondOrderModel
    number of equations: 151
    number of inputs:    1
    number of outputs:   3
    continuous-time
    second-order
    linear time-invariant
    solution_space:  NumpyVectorSpace(151)


In [4]:
pfom

SecondOrderModel(
    NumpyMatrixOperator(<151x151 sparse, 151 nnz>),
    LincombOperator(
        (NumpyMatrixOperator(<151x151 sparse, 3 nnz>),
         NumpyMatrixOperator(<151x151 sparse, 151 nnz>),
         NumpyMatrixOperator(<151x151 sparse, 451 nnz>)),
        (1.0, ProjectionParameterFunctional('a', index=0), ProjectionParameterFunctional('b', index=0))),
    NumpyMatrixOperator(<151x151 sparse, 451 nnz>),
    NumpyMatrixOperator(<151x1 dense>),
    NumpyMatrixOperator(<3x151 dense>),
    Cv=ZeroOperator(NumpyVectorSpace(3), NumpyVectorSpace(151)),
    D=ZeroOperator(NumpyVectorSpace(3), NumpyVectorSpace(1)))

In [7]:
pfom.transfer_function.eval_tf(1.j, mu={'a': 0.2, 'b': 0.2})

array([[-3.77011291e-04-1.68235310e-04j],
       [-3.99327685e-04-2.49292646e-04j],
       [ 6.29292966e-16-1.92654074e-15j]])

In [8]:
fom = pfom.with_(E = pfom.E.assemble(pfom.parameters.parse({'a':0.2, 'b':0.2})))

fom

SecondOrderModel(
    NumpyMatrixOperator(<151x151 sparse, 151 nnz>),
    NumpyMatrixOperator(<151x151 sparse, 451 nnz>),
    NumpyMatrixOperator(<151x151 sparse, 451 nnz>),
    NumpyMatrixOperator(<151x1 dense>),
    NumpyMatrixOperator(<3x151 dense>),
    Cv=ZeroOperator(NumpyVectorSpace(3), NumpyVectorSpace(151)),
    D=ZeroOperator(NumpyVectorSpace(3), NumpyVectorSpace(1)))

In [10]:
E = E0 + 0.2 * M + 0.2 * K

fom = SecondOrderModel.from_matrices(M, E, K, B, C)

print(fom)

SecondOrderModel
    class: SecondOrderModel
    number of equations: 151
    number of inputs:    1
    number of outputs:   3
    continuous-time
    second-order
    linear time-invariant
    solution_space:  NumpyVectorSpace(151, id='STATE')


<h2> Transfer function evaluation </h2>
<br/>
<font size="4">
\begin{align}
    H(s) = C(s^2 M + s E + K)^{-1} B
\end{align}
</font>

In [11]:
print(fom.transfer_function)

SecondOrderModel_transfer_function
    class: FactorizedTransferFunction
    number of inputs:  1
    number of outputs: 3
    continuous-time



In [12]:
fom.transfer_function.eval_tf(1.j)

array([[-3.77011291e-04-1.68235310e-04j],
       [-3.99327685e-04-2.49292646e-04j],
       [ 6.29292966e-16-1.92654074e-15j]])

<h2> $\mathcal{H}_\infty$ Norm</h2>
<br/>
<font size="4">
For asymptotically stable systems it holds

\begin{align}
\lVert H \rVert_{\mathcal{H}_\infty}
= \sup_{\omega \in \mathbb{R}}
\lVert H(\boldsymbol{\imath} \omega) \rVert_2.
\end{align}
</font>

In [ ]:
fom.hinf_norm()

<h3> Magnitude plot</h3>
<br/>
<font size="4">
For $\omega \in [\omega_{min}, \omega_{max}]$ plot $\lVert H(\boldsymbol{\imath} \omega) \rVert_2$.
</font>

In [ ]:
w = np.logspace(-3, 0, 100)
_ = fom.transfer_function.mag_plot(w)

<h3> Equivalent LTI system </h3>

<br/>
<br/>
<font size="4">
\begin{align}
    \begin{bmatrix}
                I & 0 \\
                0 & M
            \end{bmatrix}
            \frac{\mathrm{d}}{\mathrm{d}t}\!
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
            & =
            \begin{bmatrix}
                0 & I \\
                -K & -E
            \end{bmatrix}
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
            +
            \begin{bmatrix}
                0 \\
                B
            \end{bmatrix}
            u(t), \\
            y(t)
            & =
            \begin{bmatrix}
                C & 0
            \end{bmatrix}
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
\end{align}

  - state $[x(t), \dot{x}(t)]^T \in \mathbb{R}^{2n}$ now includes displacement and velocity
  - we can use any model reduction method for LTI systems to obtain reduced model
    
</font>

In [ ]:
lti = fom.to_lti()

print(lti)

In [ ]:
from pymor.reductors.bt import BTReductor

bt = BTReductor(lti)
bt_rom = bt.reduce(r=10)

In [ ]:
bt_err = lti - bt_rom

print(bt_err.h2_norm())

In [ ]:
hinf_err = bt_err.hinf_norm()
error_bounds = bt.error_bounds()
hsv = lti.hsv()

fig, ax = plt.subplots()
ax.semilogy(range(1, 20), error_bounds[0:19], '.-')
ax.semilogy(range(1, 20), hsv[1:20], '.-')
ax.semilogy(10, hinf_err, 'x')
_ = ax.set_title(r'Upper and lower $\mathcal{H}_\infty$ error bounds')

In [ ]:
from pymor.reductors.mt import MTReductor

mt = MTReductor(lti)
mt_rom = mt.reduce(r=10)

In [ ]:
mt_err = lti - mt_rom

print(mt_err.h2_norm())
print(mt_err.hinf_norm())

In [ ]:
from pymor.reductors.h2 import IRKAReductor

irka = IRKAReductor(lti)
irka_rom = irka.reduce(10)

In [ ]:
irka_err = lti - irka_rom

print(irka_err.h2_norm())
print(irka_err.hinf_norm())

In [ ]:
w = np.logspace(-4, 2, 100)
fig, ax = plt.subplots()

_ = bt_err.transfer_function.mag_plot(w, ax=ax, label='bt')
_ = mt_err.transfer_function.mag_plot(w, ax=ax, label='mt')
_ = irka_err.transfer_function.mag_plot(w, ax=ax, label='irka')
_ = ax.legend()

<h1> Structure preserving balanced truncation </h1>
<br/>
<font size="4">

- currently all ROMs are LTI systems
- no physical interpretation (w.r.t. position and velocity) of the ROM's state is possible
- structure preserving methods yield second-order ROM
- consider the first order representation
    
\begin{align}
    \begin{bmatrix}
                I & 0 \\
                0 & M
            \end{bmatrix}
            \frac{\mathrm{d}}{\mathrm{d}t}\!
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
            & =
            \begin{bmatrix}
                0 & I \\
                -K & -E
            \end{bmatrix}
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
            +
            \begin{bmatrix}
                0 \\
                B
            \end{bmatrix}
            u(t), \\
            y(t)
            & =
            \begin{bmatrix}
                C & 0
            \end{bmatrix}
            \begin{bmatrix}
                x(t) \\
                \dot{x}(t)
            \end{bmatrix}
\end{align}

  with the controllability Gramian

\begin{align}
    \mathbf{P} = \begin{bmatrix}
                P_p & P_{12} \\
                P_{12}^T & P_v
            \end{bmatrix}
\end{align}
    
- $P_p$ is called position controllability Gramian, $P_v$ is called velocity controllability Gramian
- similarly $Q_p$ and $Q_v$ can be obtained
- we can perform balancing of second-order system matrices with each combination of controllability and observability Gramians
</font>

In [ ]:
from pymor.reductors.sobt import *

In [ ]:
sobtp = SOBTpReductor(fom)
sobtp_rom = sobtp.reduce(r=10)
sobtp_err = fom - sobtp_rom

In [ ]:
sobtv = SOBTvReductor(fom)
sobtv_rom = sobtv.reduce(r=10)
sobtv_err = fom - sobtv_rom

In [ ]:
sobtpv = SOBTpvReductor(fom)
sobtpv_rom = sobtpv.reduce(r=10)
sobtpv_err = fom - sobtpv_rom

In [ ]:
sobtvp = SOBTvpReductor(fom)
sobtvp_rom = sobtvp.reduce(r=10)
sobtvp_err = fom - sobtvp_rom

In [ ]:
sobt = SOBTReductor(fom)
sobt_rom = sobt.reduce(r=10)
sobt_err = fom - sobt_rom

In [ ]:
w = np.logspace(-4, 2, 100)
fig, ax = plt.subplots()

_ = sobtp_err.transfer_function.mag_plot(w, ax=ax, label='sobtp')
_ = sobtv_err.transfer_function.mag_plot(w, ax=ax, label='sobtv')
_ = sobtpv_err.transfer_function.mag_plot(w, ax=ax, label='sobtpv')
_ = sobtvp_err.transfer_function.mag_plot(w, ax=ax, label='sobtvp')
_ = sobt_err.transfer_function.mag_plot(w, ax=ax, label='sobt')
_ = ax.legend()